In [1]:
import os
import re
from utils import write_text_to_file, number_to_four_digit_string, EventHandler
from config import name_of_book, client, model_to_use, backup_model, synopsis, image_genre, race_tune, image_genre_expansion

count = 1
chunk_directory = os.path.join(name_of_book,"text_chunks")
image_prompt_directory = os.path.join(name_of_book,"image_prompts")


In [ ]:
if os.path.exists(chunk_directory):
    chunk_directory_files = [f for f in os.listdir(chunk_directory) if os.path.isfile(os.path.join(chunk_directory, f))]
    # Sort files alphabetically
    chunk_directory_files.sort()
else:
    exit()

# Check if the directory exists
if not os.path.exists(image_prompt_directory):
    # Create the directory
    os.makedirs(image_prompt_directory)
    print(f"Directory '{image_prompt_directory}' was created.")
else:
    print(f"Directory '{image_prompt_directory}' already exists.")
    

In [3]:
instructions = "You are tasked with creating illustrations for specific passages from literature."
instructions += "You need to generate detailed description of illustrations that capture the essence, themes, and key elements of provided passages."
instructions += "Consider the setting, characters, action, and any symbolic elements mentioned in the text."
instructions += "Your description should be vivid enough to guide an artist in creating an artwork that complements the passage."

assistant_name = "Illustration Descriptor"

assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=instructions,
    tools=[],
    model=model_to_use,
)

assistant2 = client.beta.assistants.create(
    name=assistant_name,
    instructions=instructions,
    tools=[],
    model=backup_model,
)

def limit_context_window(chunks=[], image_prompts=[],upgrade=False):

    thread = client.beta.threads.create()
    prompt = ""
    prompt += synopsis

    if len(image_prompts) != 0:
        prompt += "The following is a record of a conversation with a previous iteration of you."
        prompt += "Use this record as a guide to create image prompts in the future."
        prompt += "Attempt to keep the style and continuity of the next illustrations the same."

    if len(image_prompts) > 2:
        prompt += "Prompt: '''" + chunks[-4] + "'''\n"
        prompt += "Response: ###" + image_prompts[-3] + "###\n"
    if len(image_prompts) > 1:
        prompt += "Prompt: '''" + chunks[-3] + "'''\n"
        prompt += "Response: ###" + image_prompts[-2] + "###\n"
    if len(image_prompts) > 0:
        prompt += "Prompt: '''" + chunks[-2] + "'''\n"
        prompt += "Response: ###" + image_prompts[-1] + "###\n"

 
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id if upgrade == False else assistant.id,
        instructions="Please follow the user's instructions and acknowlege that you understand.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )

    return thread

In [4]:
def get_image_prompt(literature, thread, count=0):
    prompt = "Based on the provided text, describe a suitable illustration that embodies the narrative, atmosphere, and significant details mentioned."
    prompt += "**Enclose the description** between triple hashes, not quotes."
    prompt += "If you were provided a record from a previous conversation, make sure the description has continuity with previous descriptions."
    prompt += "But remember that the description should be a literal description of what goes on in the image. It is meant to be a set of instructions for an artist to draw."
    prompt += "Do not simply name the characters. Instead, you should describe what the characters look like."
    prompt += "Take all the time you need to get it done right the first time. It's better to be late than incorrect."
    prompt += "Please, this is for spreading literature to the masses."
    prompt += "I encourage you to think aloud to decide the right description."
    prompt += "It should stand alone from the other descriptions, because the artist will not have access to the other descriptions."
    prompt += f"The description should be in the {image_genre} style with all characters, unless otherwise noted, being {race_tune}"
    prompt += f"This genre often uses vivid colors, exaggerated forms, and dramatic, sometimes distorted imagery to express the inner feelings of the characters or themes."
    prompt += "Finally, be as detailed as you need."
    prompt += "Text: '''" + literature + "'''"

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please follow the user's instructions.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )

    for message in messages:
        raw_response = message.content[0].text.value
        # print(raw_response)
        pattern = r'###(.*?)###'

        # Use re.search to grab the first occurrence
        match = re.search(pattern, raw_response, re.DOTALL)
        # print(match)

        if not match:
            if count % 40 == 0 and count != 0: print("\n")
            # print(raw_response)
            print("🙅",end="")
            return get_image_prompt(literature, thread, count=count+1)
        text_that_got_returned = match.group(1).strip()
        # print(text_that_got_returned)
        if len(text_that_got_returned) < 280:
            if count % 40 == 0 and count != 0: print("\n")
            print("🤏",end="") 
            return get_image_prompt(literature, thread, count=count+1)
        if text_that_got_returned[-1] not in [".","!","?",'"']:
            if count % 40 == 0 and count != 0: print("\n")
            print("🔚",end="")
            return get_image_prompt(literature, thread, count=count+1)
        # if text_that_got_returned[-1] == text_that_got_returned[-2] == ".":
        #     if count % 40 == 0 and count != 0: print("\n")
        #     print("😶",end="")
        #     return get_image_prompt(literature, count=count+1)
        break    

    return text_that_got_returned

In [ ]:
chunks = []
prompts = []
for chunked_file in chunk_directory_files:
    without_the_other_stuff = int(chunked_file.replace(name_of_book,"").replace(".txt",""))

    image_prompt_filename = name_of_book + "_imgprompt_" + number_to_four_digit_string(without_the_other_stuff) + ".txt"
    full_prompt_filepath = os.path.join(image_prompt_directory, image_prompt_filename)
    full_chunk_filepath = os.path.join(chunk_directory, chunked_file)

    with open(full_chunk_filepath, 'r', encoding='utf-8') as file:
        chunk_text = file.read()
        chunks.append(chunk_text)

    if os.path.isfile(full_prompt_filepath):
        with open(full_prompt_filepath, 'r', encoding='utf-8') as file:
            prompt_text = file.read()
            prompts.append(prompt_text)
        continue

    thread = limit_context_window(chunks, prompts)

    image_prompt_text = get_image_prompt(chunk_text, thread)
    prompts.append(image_prompt_text)

    write_text_to_file(image_prompt_directory, image_prompt_filename, image_prompt_text)

